# 🎯 Qwen2.5系列模型**LoRA/QLoRA** **微调案例**

> 说明：先用一个兼容的小模型（例如 `Qwen/Qwen2.5-1.5B-Instruct`）跑通流程，后续将 `MODEL_ID` 替换为你找到的 DeepSeek 模型仓库名即可，代码无需改动。
> 

**目标**：在单卡 A10（24GB）上，以 *小参数量* 的 DeepSeek 系列模型为例（本案例采用ModelScope来替换HuggingFace），用 **LoRA/QLoRA** 跑通一次完整的 *指令微调*（Instruction Tuning）流程。  
**硬件建议**：A10 24GB；  
**软件建议**：Python 3.10+、CUDA 12.x、PyTorch 2.3+。

---

## ✅ 本教程包括
1. LoRA/QLoRA 简介
2. 安装依赖与环境检测  
3. 选择模型与数据集（以 `Alpaca` 为经典示例）  
4. 数据预处理与 `chat_template` 适配  
5. 用 `bitsandbytes` + `peft` + `trl` 进行 **LoRA/QLoRA** 微调  
6. 保存与合并权重、推理验证  

> 注：全流程都在 **Jupyter Lab** 中逐格运行即可。

## 一、LoRA / QLoRA 简介

### LoRA（Low-Rank Adaptation）
LoRA 是一种 **轻量化模型微调方法**，它的核心思想是：  
- 在保持原始预训练模型参数 **冻结不变** 的前提下，只在部分权重矩阵（通常是 Transformer 的注意力层）上引入 **低秩矩阵分解**。  
- 用一个低秩的参数矩阵（A、B）来近似原始大矩阵的更新，从而 **大幅减少训练参数量**。  
- 优点：  
  - **参数高效**：只需训练极少量的新增参数（可低至 0.1%）。  
  - **存储友好**：多个下游任务可以共享同一个基础模型，仅保存不同任务的 LoRA 权重。  
  - **部署灵活**：推理时直接将 LoRA 权重合并到原模型，无需额外计算开销。

> 简单理解：LoRA 就像是在大模型的“固定主干”上，插入一些 **小而聪明的适配器**，让它快速学会新任务。

### QLoRA（Quantized LoRA）
QLoRA 是对 LoRA 的进一步优化，它结合了 **量化技术**，使得大模型的微调在 **单卡消费级显卡** 上也可行。  
- 核心思路：  
  1. 先将大模型的参数进行 **4-bit 量化（NF4 方案）**，降低显存占用。  
  2. 在量化后的权重上，应用 **LoRA 适配器** 进行微调。  
  3. 训练时仅更新 LoRA 层，而量化权重保持冻结。  

- 优点：  
  - **极致显存节省**：可在一张 24GB 显存的 GPU 上微调百亿参数模型。  
  - **保持性能**：量化后的 QLoRA 与全精度微调效果接近甚至相当。  
  - **实用性强**：特别适合个人开发者和中小团队。

---

### 对比总结
| 方法   | 主要手段                   | 显存消耗 | 训练参数量 | 适用场景 |
|--------|---------------------------|----------|------------|----------|
| LoRA   | 低秩矩阵分解               | 较低     | 千万级别   | 中等规模模型的高效微调 |
| QLoRA  | 量化（4-bit） + LoRA 适配 | 极低     | 千万级别   | 超大模型在消费级 GPU 上的微调 |

---

## 二、安装依赖与环境检测

### 安装依赖

### 环境版本

In [ ]:
# 📌 打印脚本相关库的版本信息
import torch, transformers, modelscope, peft

print("torch:", torch.__version__)

# transformers 是 peft 和 modelscope 依赖的核心库
try:
    import transformers
    print("transformers:", transformers.__version__)
except ImportError:
    print("transformers: 未安装")

try:
    import modelscope
    print("modelscope:", modelscope.__version__)
except ImportError:
    print("modelscope: 未安装")

try:
    import peft
    print("peft:", peft.__version__)
except ImportError:
    print("peft: 未安装")

try:
    import datasets
    print("datasets:", datasets.__version__)
except ImportError:
    print("datasets: 未安装")

try:
    import accelerate
    print("accelerate:", accelerate.__version__)
except ImportError:
    print("accelerate: 未安装")

## 三、下载模型和数据集（ModelScope版本）

### 模型下载

In [4]:
from modelscope import AutoTokenizer, AutoModelForCausalLM

model_id = "qwen/Qwen2.5-1.5B-Instruct"  # 可替换
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

UsageError: Line magic function `%modelscope` not found.


### 数据集下载

In [ ]:
from modelscope.msdatasets import MsDataset

# 选择alpaca的中文数据集
ds = MsDataset.load("alpaca-gpt4-data-zh", namespace="AI-ModelScope", split="train")


## 四、数据预处理

In [ ]:

# 自定义数据处理函数(需要针对自己的数据集范式来编写，这里只针对alpaca)
def preprocess(example):
    # 丢掉 instruction 或 output 缺失的样本
    if not example['instruction'] or not example['output']:
        return None

    # alpaca 数据有指令、输入、输出三个标签
    instruction = example['instruction']
    input_text = example.get('input') or ""  # input 可能为 None
    output_text = example['output']

    if input_text.strip():
        prompt = f"指令: {instruction}\n输入: {input_text}\n回答:"
    else:
        prompt = f"指令: {instruction}\n回答:"

    full_text = prompt + output_text

    enc = tokenizer(
        full_text,  # 需要进行token化的文本
        truncation=True,  # 文本过大的时候是否截断
        max_length=16000,  # 根据模型和数据集决定，模型的上下文, 32k甚至更大
        padding="max_length",  # 🔹 保证长度一致，DataLoader 堆叠安全
        return_tensors="pt"  # 返回的数据类型，pt:pytorch.tensor; tf:tensorflow; np:numpy
    )
    # 单个样本是字典格式
    return {
        "input_ids": enc["input_ids"][0],
        "labels": enc["input_ids"][0]
    }

train_dataset = ds.map(preprocess)
train_dataset = train_dataset.filter(lambda x: x is not None)

### 将Dataset转化成DataLoader

In [ ]:
from torch.utils.data import DataLoader, Subset

# 🛠️ 自定义批处理函数 (collate_fn)
def collate_fn(batch):
    """
    作用：
    - DataLoader 会把一个 batch 的样本（list[dict]）传进来
    - 这里需要手动拼接成 tensor，并且对齐长度（pad）
    """

    # 取出每个样本的 input_ids 和 labels，转成 tensor
    input_ids = [torch.tensor(item["input_ids"]) for item in batch]
    labels = [torch.tensor(item["labels"]) for item in batch]

    # 🔹 对 input_ids 做 padding
    #   - batch_first=True: 结果形状 (batch_size, seq_len)
    #   - padding_value=tokenizer.pad_token_id: 使用 tokenizer 的 pad_token_id 填充
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )

    # 🔹 对 labels 做 padding
    #   - 注意这里 padding_value = -100
    #   - 在 PyTorch 的 CrossEntropyLoss 里，-100 会被忽略，不参与 loss 计算
    labels = torch.nn.utils.rnn.pad_sequence(
        labels, batch_first=True, padding_value=-100
    )

    # 返回字典，方便直接喂给模型
    return {
        "input_ids": input_ids,
        "labels": labels
    }


# 📊 数据子集（仅用于测试）
# 这里为了快速验证流程，只取前 2000 条样本来训练
small_dataset = Subset(train_dataset, range(2000))

# 构建 DataLoader
train_loader = DataLoader(
    small_dataset,
    batch_size=4,        # 每次取 4 个样本
    shuffle=True,        # 打乱数据顺序
    collate_fn=collate_fn  # 使用我们自定义的 batch 拼接逻辑
)

## 五、LoRA微调

### 🔧 配置 LoRA 训练参数

```python
lora_config = LoraConfig(
    r=4,                          
    lora_alpha=16,               
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05,             
    bias="none",               
    task_type="CAUSAL_LM"         
)
```

r=4
- 表示低秩矩阵的秩值（rank），值越大 → 适配能力更强 → 参数量也随之增加。  
- 这里选择 `4`，意味着 **轻量级训练**，适合小规模任务或快速实验。  

lora_alpha=16
- 缩放因子，用于调整 LoRA 的输出幅度。  
- 一般经验是 **lora_alpha ≈ 2 × r**，所以这里 `16` 配合 `r=4` 是合理的。  

target_modules=["q_proj", "v_proj"]
- LoRA 只在注意力机制的 **Query** 和 **Value** 投影层中生效。  
- 这是最常见的设置，既保证效果，又控制参数量。  

lora_dropout=0.05
- 在 LoRA 层中添加 **5% 的 dropout**，提升泛化能力。  
- 数据量很大时可以调低到 `0`；数据少时可以适当调高（如 `0.1`）。  

bias="none"
- 不训练 bias 参数，保证模型轻量化。  
- 大多数场景下用 `"none"` 即可。  

task_type="CAUSAL_LM"
- 表示任务是 **自回归语言建模**（比如 Qwen、GPT 类模型）。  
- 必须和任务类型一致，否则 forward 过程会报错。  

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=4,            
    lora_alpha=16,                
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05,             
    bias="none",                    
    task_type="CAUSAL_LM"           
)

# 🚀 将基础模型包装为 PEFT 模型
model = get_peft_model(model, lora_config)

# 打印当前可训练参数量（仅 LoRA 部分），其余参数被冻结
model.print_trainable_parameters()

# 训练超参数
num_train_epochs = 2  # 

### 训练

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-4)

model.train()
for epoch in range(num_train_epochs):
    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(model.device)
        labels = batch["labels"].to(model.device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss··
        loss.backward()
        optimizer.step()

        if step % 100 == 0:   # 每 100 个 step 打印一次
            print(f"Epoch {epoch} | Step {step} | Loss {loss.item():.4f}")